In [1]:
%load_ext autoreload
%autoreload 2
import subprocess
import math
import numpy as np
import pandas as pd 
from pathlib import Path
import lightgbm as lgb
import queue


def update_weight():                                                        
    """Create a callback that generates the weight for each query                                      
    Returns                                                                                            
    -------                                                                                            
    callback : function                                                                                
        The callback that prints the evaluation results every ``period`` iteration(s).                 
    """                    
    #Modifying the train set in the mbooster model
    def _callback(env):
        #set the weight if it is less than -2
        weight = []
        for result in env.risk_reward_list:
            if (result[0] == 'train'):
                flt = [x for x in result[2] if x < -2.0]
                print('Number of docs that is risky (<-2.0): ')
                print(len(flt))
                for idx, rr in enumerate(result[2]):
                    if (rr< -2.0):
                        weight += env.model.train_set.group[idx] * [1.0]
                    else:
                        weight += env.model.train_set.group[idx] * [0.8]
        weights = np.array(weight)
        env.model.train_set.weight = weights
    _callback.order = 10
    return _callback

class riskrewardUtil:

    bpath = "/research/remote/petabyte/users/robert/LightGBM/Experiments/"
    temppath = bpath + "results/evalMetrics/"

    qidvalid = qrelsvalid = baselinename = baselineeval = ""
    qidtrain = qrelstrain = baselinetrainname = baselinetraineval = ""

    # parameterized constructor 
    def __init__(self, valqid, valqrels, baselinerun, baselineeval, 
                 qidtrain, qrelstrain, baselinetrainname, baselinetraineval): 
        self.qidvalid = valqid
        self.qrelsvalid = valqrels
        self.baselinename = baselinerun
        self.baselineeval = baselineeval
        self.qidtrain = qidtrain
        self.qrelstrain = qrelstrain
        self.baselinetrainname = baselinetrainname
        self.baselinetraineval = baselinetraineval
        
    
    def evalScore(self, file, output, qrels, depth=5, bpath=bpath):
        bashCommand = "ls"
        script = "/research/remote/petabyte/users/robert/Utilities/grdeval/target/release/grdeval"
        countndcg = subprocess.run(script + ' -k %d %s %s | head -n -1 > %s' % (depth, qrels, file, output), shell=True, check=True, cwd = bpath)

    #Generate runfile based on the model (Predicting), getting the model, output, test_file, and qid
    #test_file = Features to predict (X file in svm light file) in numpy array
    #QID, give the qid in numpy array that is generated by load svm light file
    def predictgenerateRunFile(self, model, output, test_file, qid):
        bst = lgb.Booster(model_file=model)
        y_pred = bst.predict(test_file)

        #Script automation version
        result = pd.DataFrame(qid)

        #result['Pred'] = y_test << for generating qrel file
        result['Pred'] = y_pred

        #Append necessary column for trec eval
        result['model'] = 'LTR'
        result.insert(1, '2nd', 'Q0')

        #Get counts for each of the query ID and put them as the index
        qidCount = result.groupby(0)
        arr = pd.Series()
        for x in qidCount:
            #print(len(x[1]))
            arr = arr.append(pd.Series(range(1, len(x[1]) + 1)))
        arr = arr.reset_index(drop=True)
        result.insert(2, 'index', arr)

        #Create the Doc_ID for each of the query
        result.insert(2,'Doc_id', result[0].map(str) + "." + result['index'].map(str))

        #New Sorting version
        result = result.sort_values([0,'Pred'], ascending=[1,0]).reset_index(drop=True)

        #Reinsert the index
        result = result.drop(columns=['index'], axis=0)
        result.insert(3,'index',arr)

        result.to_csv(output, index=False, header=None, sep=' ')
        return result
        

    #Generating run file after we have the prediction
    def generateRunFile(self, pred, output, qid=qidvalid):
        #Script automation version
        result = pd.DataFrame(qid)

        #result['Pred'] = y_test << for generating qrel file
        result['Pred'] = pred

        #Append necessary column for trec eval
        result['model'] = 'LTR'
        result.insert(1, '2nd', 'Q0')

        #Get counts for each of the query ID and put them as the index
        qidCount = result.groupby(0)
        arr = pd.Series()
        for x in qidCount:
            #print(len(x[1]))
            arr = arr.append(pd.Series(range(1, len(x[1]) + 1)))
        arr = arr.reset_index(drop=True)
        result.insert(2, 'index', arr)

        #Create the Doc_ID for each of the query
        result.insert(2,'Doc_id', result[0].map(str) + "." + result['index'].map(str))

        #New Sorting version
        result = result.sort_values([0,'Pred'], ascending=[1,0]).reset_index(drop=True)

        #Reinsert the index
        result = result.drop(columns=['index'], axis=0)
        result.insert(3,'index',arr)

        result.to_csv(output, index=False, header=None, sep=' ')

    # f(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
    def trisk1(self, pred, train_data):

        # Globals
        alpha=1.0
        run_map = pred
        risk_reward = []
        baseline_map = []

        #Get bm25 features
        for row in range(0, train_data.data.shape[0]):
            baseline_map.append(train_data.data[row,109])

        runname = self.temppath + 'runfile'
        runeval = self.temppath + 'runeval'

        self.generateRunFile(run_map, runname, self.qidvalid)
        self.evalScore(runname, runeval, self.qrelsvalid)

        #check if baseline exist to generate only once

        checkfile = Path(self.baselinename)
        if(not checkfile.exists()):
            self.generateRunFile(baseline_map, self.baselinename, self.qidvalid)
            self.evalScore(self.baselinename, self.baselineeval, self.qrelsvalid)

        try:
            run_map = pd.read_csv(runeval)
            run_map = run_map['ndcg@5']
            baseline_map = pd.read_csv(self.baselineeval)
            baseline_map = baseline_map['ndcg@5']
        except:
            print("No Eval file found!")

        urisk = 0.0
        c = len(run_map)

        # Calculations from Dincer TRisk paper.
        def risk_reward_tradeoff_score(topic):
            r = run_map[topic]
            b = baseline_map[topic]

            if (r>b):
                return float((r - b))
            elif (r<b):
                return float((1 + alpha) * (r - b))
            else:
                return 0.000


        def sx():
            sum = 0.0
            for x in risk_reward:
                sum += ((x - urisk) ** 2)
            sum /= c
            return math.sqrt(sum)

        def parametric_standard_error_estimation():
            temp = sx()
            return (1.0 / math.sqrt(c)) * temp


        for index, topic in enumerate(run_map):
            val = risk_reward_tradeoff_score(index)
            risk_reward.append(val)

        # Calculate the mean of the risk reward scores. This is the URisk score.

        urisk = float(sum(risk_reward)) / float(len(risk_reward))

        # Calculate TRisk score
        se = parametric_standard_error_estimation()
        trisk = urisk / se

        return "Trisk alpha 1", float(trisk), True
    
    #Only use with the custom callback!
    #TODO: pass riskreward score to callback and generate the weight from there
    def riskrewardWeighted(self, pred, train_data):

        # Globals
        alpha=1.0
        run_map = pred
        risk_reward = []
        baseline_map = []
        score_per_topic = []
        

        #Get bm25 features
        for row in range(0, train_data.data.shape[0]):
            baseline_map.append(train_data.data[row,636])

        #Generate temp runfile and runeval
        runname = self.temppath + 'runfile'
        runeval = self.temppath + 'runeval'
        
        #It means this is a valid data because valid data is referenced to train data
        if (train_data.reference != None):
            qrels = self.qrelsvalid
            qid = self.qidvalid
            baselinename = self.baselinename
            baselineeval = self.baselineeval
        #It means this is a train data
        else:
            qrels = self.qrelstrain
            qid = self.qidtrain
            baselinename = self.baselinetrainname
            baselineeval = self.baselinetraineval
        
        #Generate each ndcg score evaluated
        self.generateRunFile(run_map, runname, qid)
        self.evalScore(runname, runeval, qrels)

        #check if baseline exist to generate only once
        checkfile = Path(baselinename)
        if(not checkfile.exists()):
            self.generateRunFile(baseline_map, baselinename, qid)
            self.evalScore(baselinename, baselineeval, qrels)

        run_map = pd.read_csv(runeval)
        run_map = run_map['ndcg@5']
        baseline_map = pd.read_csv(baselineeval)
        baseline_map = baseline_map['ndcg@5']
    
        urisk = 0.0
        c = len(run_map)

        # Calculations from Dincer TRisk paper.
        def risk_reward_tradeoff_score(topic):
            r = run_map[topic]
            b = baseline_map[topic]

            if (r>b):
                return float((r - b))
            elif (r<b):
                return float((1 + alpha) * (r - b))
            else:
                return 0.000

        def sx():
            sum = 0.0
            for x in risk_reward:
                sum += ((x - urisk) ** 2)
            sum /= c
            return math.sqrt(sum)

        def parametric_standard_error_estimation():
            temp = sx()
            return (1.0 / math.sqrt(c)) * temp

        for index, topic in enumerate(run_map):
            val = risk_reward_tradeoff_score(index)
            risk_reward.append(val)

        # Calculate the mean of the risk reward scores. This is the URisk score.
        urisk = float(sum(risk_reward)) / float(len(risk_reward))
        
        sxval = sx()
        for rr in risk_reward:
            score_per_topic.append(rr/sxval)
        
        
        #Read qrels, return index of query that is not evaluated on zeroindex
        temp = pd.read_csv(qrels, sep=' ', header=None)
        temp = temp.groupby(0)
        counter = 0
        #Contains index of query that is not evaluated
        zeroindex = []
        for idx, x in enumerate(temp):
            #Look for 0 value in the qrels
            if (not np.any(x[1][3])):
                counter+=1
                zeroindex.append(idx)
        
        #Filling the empty query to each of the index get from above
        a = np.arange(len(temp))
        #Fill each index of unlabelled query with 0 (unjudged)
        for x in zeroindex:
            a[x] = 0
        a[0] = 1
        counter = 0
        #Fill the rest with the score of the topic
        for y in score_per_topic:
            if (a[counter] != 0):
                a[counter] = y
                counter += 1
            else:
                while(a[counter] == 0):
                    counter+=1
                a[counter] = y
                counter+=1
        
        score_per_topic = a.tolist()
        
        # Calculate TRisk score
        se = parametric_standard_error_estimation()
        trisk = urisk / se
        
        #final score is the trisk value
        score_per_topic.append(trisk)
            
        return "Risk reward score: ", score_per_topic, True

In [15]:
import lightgbm as lgb
import numpy as np
import sklearn
import pandas as pd
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import mean_squared_error
import riskrewardutil as rru

basePath = '/research/remote/petabyte/users/robert/Utilities/ltr-baseline/mslr10k/'
expPath = "/research/remote/petabyte/users/robert/LightGBM/Experiments/"
dataPath = basePath + 'dat/MSLR-WEB10K/'
modelPath = basePath + 'model/'
runPath = basePath + 'run/'
qrelsFile = dataPath + '../all.test.qrels'


#Count the amount of queries 
def group_counts(arr):
    d = np.ones(arr.size, dtype=int)
    d[1:] = (arr[:-1] != arr[1:]).astype(int)
    return np.diff(np.where(np.append(d, 1))[0])

name = "lgbm.1500.63.0.05.0.4.withTrisk"

combineddf = pd.DataFrame()
earlystop = [50, 150, 300, 500]

for stop in earlystop:
    combineddf = pd.DataFrame()
    name = name + '.earlystop%d' % (stop)
    for fold in range(1,6):
        suffix = name + ".fold%d" % (fold)

        X, y, qid = load_svmlight_file(dataPath + 'Fold%d/train.txt' % (fold), query_id=True)
        train_data = lgb.Dataset(X, label=y, group=group_counts(qid), free_raw_data=False)
        X_valid, y_valid, qidValid = load_svmlight_file(dataPath + 'Fold%d/vali.txt' % (fold), query_id=True)
        valid_data = lgb.Dataset(X_valid, label=y_valid, group=group_counts(qidValid), free_raw_data=False)
        X_test, y_test, qid = load_svmlight_file(dataPath + 'Fold%d/test.txt' % (fold), query_id=True)
        test_data = lgb.Dataset(X_test, label=y_test, group=group_counts(qid), free_raw_data=False)

        #Global variables needed for custom metrics, qid and qrels for each valid file

        qidvalid= qidValid
        qrelsvalid = dataPath + 'Fold%d/vali.qrels' % (fold)
        #Another global variables containing bm25 features for each fold
        baselinename = 'resultsmslr10k/evalMetrics/baselinevalrun%d' % (fold)
        baselineeval = 'resultsmslr10k/evalMetrics/baselinevaleval%d' % (fold)

        metrics = rru.riskrewardUtil(qidvalid, qrelsvalid, baselinename, baselineeval)

        #Setup Param File and generate different models for hyper parameter tuning
        param = {'num_leaves':63, 'num_trees':1500, 'objective':'lambdarank',
             'learning_rate': 0.05,'feature_fraction': 0.4,
             'bagging_fraction': 0.8,'bagging_freq': 5,
             'verbose': 1, 'early_stopping_rounds': 150}

        param['metric'] = 'None'
        #Train Model
        num_round = 10
        bst = lgb.train(param, train_data, num_round, valid_sets=[valid_data], feval=metrics.trisk1)
        bst.save_model(modelPath + suffix)

        combineddf = combineddf.append(metrics.predictgenerateRunFile(modelPath + suffix, runPath + suffix, X_test, qid))


    combineddf.to_csv(runPath + 'run.all.' + name, index=False, header=None, sep=' ')
    #TO DO 
    #Get all the run file as 1 file, then eval them with evalScore
    metrics.evalScore(runPath + 'run.all.' + name, expPath + 'resultsmslr10k/evalPerQueryLGBMTriskes%d' % (stop), qrelsFile)




In [22]:
valid_data.data

<71083x699 sparse matrix of type '<class 'numpy.float64'>'
	with 15816715 stored elements in Compressed Sparse Row format>

In [21]:
len(test_data.group)

6983

## Testing on yahoo for faster purpose

In [3]:
import lightgbm as lgb
import numpy as np
import sklearn
import pandas as pd
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import mean_squared_error

dataPath = '/research/remote/collections/learning_to_rank/LTRv2/'

#Count the amount of queries 
def group_counts(arr):
    d = np.ones(arr.size, dtype=int)
    d[1:] = (arr[:-1] != arr[1:]).astype(int)
    return np.diff(np.where(np.append(d, 1))[0])

dataPaths = [dataPath + 'set1.train.txt', dataPath + 'set1.valid.txt', dataPath + 'set1.test.txt']

def load_lgb_data(dataPaths):
    datas = []
    for data in dataPaths:
        X, y, qid = load_svmlight_file(data, query_id=True)
        datas.append(lgb.Dataset(X, label=y, group=group_counts(qid), free_raw_data=False))
    return datas


train_data, valid_data, test_data = load_lgb_data(dataPaths)

#new
valid_data.reference = train_data

#Reload the test set to get X and Y to predict
X_test, y_test, qid = load_svmlight_file(dataPath + 'set1.test.txt', query_id=True)

#Create Valid set to predict and see the score
X_valid, y_valid, qidValid = load_svmlight_file(dataPath + 'set1.valid.txt', query_id=True)

X_train, y_train, qidTrain = load_svmlight_file(dataPath + 'set1.train.txt', query_id=True)

In [ ]:
#Setup Param File and generate different models for hyper parameter tuning
bpath = "/research/remote/petabyte/users/robert/LightGBM/Experiments/"
qrelsvalid = "qrels.yahooValid"
baselinename = 'results/evalMetrics/baselinerun'
baselineeval = 'results/evalMetrics/baselineeval'
qrelstrain = "qrels.yahooTrain"
baselinetrainname = 'results/evalMetrics/baselinetrainrun'
baselinetraineval = 'results/evalMetrics/baselinetraineval'

param = {'num_leaves':63, 'num_trees':2000, 'objective':'lambdarank',
     'learning_rate': 0.1,'feature_fraction': 0.4,
     'bagging_fraction': 0.8,'bagging_freq': 5,
     'verbose': 1, 'early_stopping_rounds': 175, 'first_metric_only': True}

util = riskrewardUtil(qidValid, bpath + qrelsvalid, baselinename, baselineeval, 
                     qidTrain, bpath + qrelstrain, baselinetrainname, baselinetraineval)

callbacks = set()
callbacks.add(update_weight())


param['metric'] = 'None'
#Train Model
num_round = 10
bst = lgb.train(param, train_data, num_round, valid_sets=[valid_data, train_data],
                valid_names=['valid','train'], feval=util.riskrewardWeighted, callbacks=callbacks)
bst.save_model('modelsP05LR/2000.63.0.4.withTrisk.txt')

predictgenerateRunFile('modelsP05LR/2000.63.0.4.withTrisk.txt', 'predictions05LR/2000.63.0.4.withTrisk.txt')

evalScore('predictions05LR/2000.63.0.4.withTrisk.txt', 'results/evalPerQueryLGBMTrisk', qrels='qrels.yahooTest')

/home/sh1/S3713561/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:119: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/sh1/S3713561/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:124: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Number of docs that is risky (<-2.0): 
73
[1]	train's Risk reward score: : 66.2503	valid's Risk reward score: : -11.7608
Training until validation scores don't improve for 175 rounds.
Number of docs that is risky (<-2.0): 
86
[2]	train's Risk reward score: : 69.7076	valid's Risk reward score: : -7.79926
Number of docs that is risky (<-2.0): 
80
[3]	train's Risk reward score: : 71.3925	valid's Risk reward score: : -7.04822
Number of docs that is risky (<-2.0): 
83
[4]	train's Risk reward score: : 73.1436	valid's Risk reward score: : -5.37791
Number of docs that is risky (<-2.0): 
77
[5]	train's Risk reward score: : 74.3893	valid's Risk reward score: : -4.71592
Number of docs that is risky (<-2.0): 
82
[6]	train's Risk reward score: : 75.7396	valid's Risk reward score: : -4.36795
Number of docs that is risky (<-2.0): 
80
[7]	train's Risk reward score: : 76.1688	valid's Risk reward score: : -3.95786
Number of docs that is risky (<-2.0): 
75
[8]	train's Risk reward score: : 77.1137	valid's

Number of docs that is risky (<-2.0): 
60
[68]	train's Risk reward score: : 102.558	valid's Risk reward score: : 1.39108
Number of docs that is risky (<-2.0): 
58
[69]	train's Risk reward score: : 103.054	valid's Risk reward score: : 1.46757
Number of docs that is risky (<-2.0): 
58
[70]	train's Risk reward score: : 103.096	valid's Risk reward score: : 1.71134
Number of docs that is risky (<-2.0): 
58
[71]	train's Risk reward score: : 103.282	valid's Risk reward score: : 1.7238
Number of docs that is risky (<-2.0): 
57
[72]	train's Risk reward score: : 103.48	valid's Risk reward score: : 1.74482
Number of docs that is risky (<-2.0): 
54
[73]	train's Risk reward score: : 103.936	valid's Risk reward score: : 1.78654
Number of docs that is risky (<-2.0): 
55
[74]	train's Risk reward score: : 104.119	valid's Risk reward score: : 1.65961
Number of docs that is risky (<-2.0): 
54
[75]	train's Risk reward score: : 104.183	valid's Risk reward score: : 1.66005
Number of docs that is risky (<-2.

Number of docs that is risky (<-2.0): 
50
[136]	train's Risk reward score: : 116.692	valid's Risk reward score: : 4.12119
Number of docs that is risky (<-2.0): 
50
[137]	train's Risk reward score: : 116.93	valid's Risk reward score: : 4.05732
Number of docs that is risky (<-2.0): 
49
[138]	train's Risk reward score: : 117.095	valid's Risk reward score: : 4.13476
Number of docs that is risky (<-2.0): 
48
[139]	train's Risk reward score: : 117.293	valid's Risk reward score: : 4.11691
Number of docs that is risky (<-2.0): 
48
[140]	train's Risk reward score: : 117.398	valid's Risk reward score: : 4.04385
Number of docs that is risky (<-2.0): 
48
[141]	train's Risk reward score: : 117.624	valid's Risk reward score: : 4.15149
Number of docs that is risky (<-2.0): 
48
[142]	train's Risk reward score: : 117.778	valid's Risk reward score: : 4.10884
Number of docs that is risky (<-2.0): 
49
[143]	train's Risk reward score: : 117.985	valid's Risk reward score: : 4.122
Number of docs that is risk

Number of docs that is risky (<-2.0): 
47
[204]	train's Risk reward score: : 128.994	valid's Risk reward score: : 4.69614
Number of docs that is risky (<-2.0): 
46
[205]	train's Risk reward score: : 129.167	valid's Risk reward score: : 4.72151
Number of docs that is risky (<-2.0): 
46
[206]	train's Risk reward score: : 129.35	valid's Risk reward score: : 4.68761
Number of docs that is risky (<-2.0): 
46
[207]	train's Risk reward score: : 129.368	valid's Risk reward score: : 4.78054
Number of docs that is risky (<-2.0): 
46
[208]	train's Risk reward score: : 129.479	valid's Risk reward score: : 4.82195
Number of docs that is risky (<-2.0): 
46
[209]	train's Risk reward score: : 129.619	valid's Risk reward score: : 4.81192
Number of docs that is risky (<-2.0): 
45
[210]	train's Risk reward score: : 129.97	valid's Risk reward score: : 4.8122
Number of docs that is risky (<-2.0): 
45
[211]	train's Risk reward score: : 130.087	valid's Risk reward score: : 4.74714
Number of docs that is risk

In [246]:
#Testing the array filling
import random
a = np.arange(20)
riskreward = [random.randint(1,10) for _ in range(16)]
index = [5,6,18,19]
for x in index:
    a[x] = 0
a[0] = 1
counter = 0
for y in riskreward:
    if (a[counter] != 0):
        a[counter] = y
        counter += 1
    else:
        #print(counter)
        while(a[counter] == 0):
            counter+=1
        a[counter] = y
        counter+=1
a

array([ 1,  3,  8,  5, 10,  0,  0,  8, 10,  5,  1,  7,  2,  8,  3,  1,  2,
        6,  0,  0])

In [6]:
a = []
a += 5 * [1.0]
a

[1.0, 1.0, 1.0, 1.0, 1.0]

In [7]:
a += 5 * [0.8]
a

[1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.8, 0.8, 0.8, 0.8]

In [15]:
def numarray(a):
    b = np.array(a)
    b = b[b<1.0]
    print(len(b))

In [16]:
def listcomprehension(a):
    my_list = [x for x in a if x < 1.0]
    print(len(my_list))

In [17]:
%timeit numarray(a)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [18]:
%timeit listcomprehension(a)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [3]:
test = pd.read_csv('/home/sh1/S3713561/work/Utilities/ltr-baseline/mq2009/run/cw09b-bm25-10k-1-50.run', 
                   sep=' ', header=None)
filtered = test.groupby(0)


Exception: Column(s) 0 already selected

In [10]:
for idx, x in enumerate(filtered):
    print(x)

(1,       0   1                          2      3         4      5
0     1  Q0  clueweb09-en0011-02-12744      1  15.83570  indri
1     1  Q0  clueweb09-en0010-57-32937      2  15.81040  indri
2     1  Q0  clueweb09-en0007-79-21751      3  15.52880  indri
3     1  Q0  clueweb09-en0010-57-32259      4  15.51990  indri
4     1  Q0  clueweb09-en0010-79-02218      5  15.50260  indri
5     1  Q0  clueweb09-en0001-02-21241      6  15.44480  indri
6     1  Q0  clueweb09-en0010-57-32586      7  14.77400  indri
7     1  Q0  clueweb09-enwp00-63-13044      8  14.72910  indri
8     1  Q0  clueweb09-enwp01-49-16274      9  14.71200  indri
9     1  Q0  clueweb09-enwp01-59-16163     10  14.71180  indri
10    1  Q0  clueweb09-enwp02-06-15081     11  14.71140  indri
11    1  Q0  clueweb09-enwp00-83-00422     12  14.71070  indri
12    1  Q0  clueweb09-enwp01-78-05310     13  14.70890  indri
13    1  Q0  clueweb09-enwp00-39-09864     14  14.70890  indri
14    1  Q0  clueweb09-enwp00-26-05310     15  14.7

[10000 rows x 6 columns])
(17,          0   1                          2      3        4      5
159415  17  Q0  clueweb09-en0010-76-06077      1  16.7134  indri
159416  17  Q0  clueweb09-en0010-33-12406      2  16.7035  indri
159417  17  Q0  clueweb09-en0003-13-11758      3  16.7013  indri
159418  17  Q0  clueweb09-en0010-35-03887      4  16.6875  indri
159419  17  Q0  clueweb09-en0008-89-11224      5  16.6835  indri
159420  17  Q0  clueweb09-en0002-75-20040      6  16.6813  indri
159421  17  Q0  clueweb09-en0011-72-05148      7  16.6811  indri
159422  17  Q0  clueweb09-en0008-89-11091      8  16.6802  indri
159423  17  Q0  clueweb09-en0002-45-31126      9  16.6798  indri
159424  17  Q0  clueweb09-en0008-89-11194     10  16.6787  indri
159425  17  Q0  clueweb09-en0003-27-21155     11  16.6785  indri
159426  17  Q0  clueweb09-en0008-89-11225     12  16.6742  indri
159427  17  Q0  clueweb09-en0004-90-24859     13  16.6728  indri
159428  17  Q0  clueweb09-en0011-97-00285     14  16.6719  

[10000 rows x 6 columns])
(33,          0   1                          2      3        4      5
319415  33  Q0  clueweb09-en0007-27-13972      1  17.7112  indri
319416  33  Q0  clueweb09-en0010-50-37430      2  17.7097  indri
319417  33  Q0  clueweb09-en0007-28-28066      3  17.6994  indri
319418  33  Q0  clueweb09-en0006-23-33808      4  17.6905  indri
319419  33  Q0  clueweb09-en0006-41-10193      5  17.6896  indri
319420  33  Q0  clueweb09-en0004-10-03867      6  17.6883  indri
319421  33  Q0  clueweb09-en0001-05-27684      7  17.6872  indri
319422  33  Q0  clueweb09-en0003-85-29611      8  17.6867  indri
319423  33  Q0  clueweb09-en0007-38-09277      9  17.6857  indri
319424  33  Q0  clueweb09-en0001-84-08947     10  17.6851  indri
319425  33  Q0  clueweb09-en0005-55-20306     11  17.6834  indri
319426  33  Q0  clueweb09-en0007-68-01959     12  17.6833  indri
319427  33  Q0  clueweb09-en0006-69-21496     13  17.6821  indri
319428  33  Q0  clueweb09-en0004-67-00302     14  17.6803  

In [19]:
temp.head(n=10)

,0,1,2,3,4,5
479550,50,Q0,clueweb09-en0001-68-17416,1,13.4340,indri
479551,50,Q0,clueweb09-en0009-71-20318,2,13.4335,indri
479552,50,Q0,clueweb09-en0010-52-03560,3,13.4254,indri
479553,50,Q0,clueweb09-en0006-99-22471,4,13.3559,indri
479554,50,Q0,clueweb09-en0006-35-10291,5,13.2837,indri
479555,50,Q0,clueweb09-en0002-19-09964,6,13.2582,indri
479556,50,Q0,clueweb09-en0005-28-10983,7,13.2141,indri
479557,50,Q0,clueweb09-en0010-43-24424,8,13.2079,indri
479558,50,Q0,clueweb09-en0000-17-15085,9,13.1332,indri
479559,50,Q0,clueweb09-en0006-98-08062,10,13.1221,indri


In [22]:
temp.head(n=10)[3].iloc[5]

6

In [40]:
bpath1 = '/research/remote/petabyte/users/robert/Utilities/ltr-baseline/mq2009/bestcutofflgbm/'
bpath2 = '/home/sh1/S3713561/work/Utilities/ltr-baseline/cw09b/bestcutofflgbm/'
tempdf = pd.DataFrame(columns=['1-50','51-100','101-150','151-200'])

In [62]:
file = ['optimalk1-50', 'optimalk51-100', 'optimalk101-150', 'optimalk151-200']


data = []
for x in file:
    temp = pd.read_csv(bpath1 + 'threshold0.9/' + x, header=None)
    data.append(temp[1].mean())

tempdf.loc[9] = data

In [63]:
processed = tempdf
processed = processed.applymap(int)
processed['medrbp'] = [0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,0.9]

In [64]:
processed

,1-50,51-100,101-150,151-200,medrbp
0,1103,1542,2477,1118,0.001
1,556,888,945,461,0.100
2,392,694,557,346,0.200
3,296,596,381,284,0.300
4,236,471,317,251,0.400
5,183,399,182,196,0.500
6,147,331,151,161,0.600
7,112,263,136,129,0.700
8,88,186,129,101,0.800
9,65,138,77,62,0.900


In [74]:
cw09b = pd.DataFrame()
k001 = pd.read_csv(bpath2 + 'optimalk001', header=None)
data = [k001[1].mean()]

In [75]:
for y in range(1,10):
    temp = pd.read_csv(bpath2 + 'threshold0.%d/' % (y) + 'optimalk1-50', header=None)
    data.append(temp[1].mean())

cw09b['averagecutoff'] = data
cw09b = cw09b.applymap(int)
cw09b['medrbp'] = [0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,0.9]
cw09b

,averagecutoff,medrbp
0,1284,0.001
1,690,0.100
2,505,0.200
3,366,0.300
4,287,0.400
5,223,0.500
6,181,0.600
7,144,0.700
8,105,0.800
9,77,0.900


In [77]:
processed['1ndcg20'] = [0.3234, 0.3207, 0.3246, 0.3195, 0.3047, 0.2952, 0.2917, 0.2811, 0.2560, 0.2458]
processed['51ndcg20'] = [0.2488, 0.2536, 0.2475, 0.2466, 0.2230, 0.2112, 0.1970, 0.1720, 0.1514, 0.1421]
processed['101ndcg20'] = [0.2135, 0.2163, 0.2166, 0.2086, 0.2022, 0.2028, 0.1978, 0.1974, 0.1892, 0.1857]
processed['151ndcg20'] = [0.1493, 0.1513, 0.1525, 0.1531, 0.1467, 0.1467, 0.1330, 0.1302, 0.1223, 0.1184]

In [79]:
processed.to_csv('mq2009optcutoff.csv', index=False)

In [80]:
cw09b['ndcg20'] = [0.2557,0.2645,0.2766,0.2716,0.2655,0.2655,0.2562,0.2579,0.2497,0.2358]
cw09b

,averagecutoff,medrbp,ndcg20
0,1284,0.001,0.2557
1,690,0.100,0.2645
2,505,0.200,0.2766
3,366,0.300,0.2716
4,287,0.400,0.2655
5,223,0.500,0.2655
6,181,0.600,0.2562
7,144,0.700,0.2579
8,105,0.800,0.2497
9,77,0.900,0.2358


In [81]:
cw09b.to_csv('cw09boptcutoff.csv', index=False)